In [1]:

"""
This is the script to simulate the nonlinear behavior of the 35 models that are calibrated on the behavior of the
reference strain, trpD9923 and the recombinant strain, trpD9923/pJLaroGfbr.
"""
import numpy as np
import pandas as pd
from configparser import ConfigParser
import time
from pytfa.io.json import load_json_model
from pytfa.optim.constraints import *

from skimpy.io.yaml import load_yaml_model
from skimpy.io.regulation import load_enzyme_regulation
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, load_concentrations, load_equilibrium_constants
from skimpy.core.parameters import load_parameter_population
from skimpy.core.solution import ODESolutionPopulation
from skimpy.utils.tabdict import TabDict
from skimpy.utils.namespace import NET, QSSA
from skimpy.simulations.reactor import make_batch_reactor

import os
import glob
import time as T
import matplotlib.pyplot as plt
from sys import argv


''' Run parameters and paths '''
# Cellular parameters
CONCENTRATION_SCALING = 1e9  # 1 mol to 1 mmol
TIME_SCALING = 1  # 1hr
DENSITY = 1105  # g/L
GDW_GWW_RATIO = 0.3  # Assumes 70% Water
flux_scaling_factor = 1e-3 * (GDW_GWW_RATIO * DENSITY) * CONCENTRATION_SCALING / TIME_SCALING

# Ode simulation parameters
TOTAL_TIME = 60
N_STEPS = 1000


In [2]:
repeat = 10
generation = 23

In [3]:
# parse arguments from configfile
configs = ConfigParser()
configs.read('bioreactor_configfile.ini')

base = configs['PATHS']['base']
met_model = configs['PATHS']['met_model']
model_file = configs['PATHS']['model_file']
thermo_experiment_file = configs['PATHS']['thermo_experiment_file']
regulation_file = configs['PATHS']['regulation_file']
kinetic_params_file = configs['PATHS']['kinetic_params_file']
steady_states_file = configs['PATHS']['steady_states_file']
generated_parameter_name_file = configs['PATHS']['generated_parameter_names']
output_folder = configs['BIOREACTOR_SIMULATION']['output_dir']
steady_states_sample = int(configs['BIOREACTOR_SIMULATION']['steady_states_sample'])


path_to_kmodel = f'{base}/{met_model}/kinetic/{model_file}'
path_to_tmodel = f'{base}/{met_model}/thermo/{thermo_experiment_file}'
path_to_params = f'{base}/{met_model}/parameters/gen_{repeat}_{generation}_parameters.hdf5'
path_to_regulation_data = f'{base}/{met_model}/{regulation_file}'
path_to_tfa_samples = f'{base}/{met_model}/steady_state_samples/{steady_states_file}'


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:

'''
Model and simulation initiation + preparation
'''

# Load models
tmodel = load_json_model(path_to_tmodel)
kmodel_draft = load_yaml_model(path_to_kmodel)

# Add regulation data to kinetic model
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel_draft.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel_draft.reactants.keys()))]
kmodel = load_enzyme_regulation(kmodel_draft, df_regulations_all)


2024-03-11 16:19:14,265 - thermomodel_new - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2024-03-11 16:19:17,395 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [5]:
k_names = []
k_regulation = []

for k, v in kmodel.parameters.items():
    if k.startswith("km_") or "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
        if k not in k_names:
            k_names.append(k)
            
        if "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
            k_regulation.append(k)

print(f"The number of generated k in model is {len(k_names)}, \nand the number of regulation parameters is {len(k_regulation)}.")

The number of generated k in model is 464, 
and the number of regulation parameters is 106.


In [6]:
# Load set of parameters corresponding to the 35 kinetic models
parameter_population = load_parameter_population(path_to_params)

# Reactor initialization
reactor = make_batch_reactor('single_species.yaml', df_regulation= df_regulations_all)
reactor.compile_ode(add_dilution=False)
reactor_volume = reactor.models.strain_1.parameters.strain_1_volume_e.value

kinetic_models = list(parameter_population._index.keys())

2024-03-11 16:19:22,213 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [8]:

sols_wt= []
for this_model in kinetic_models:
    print("\nRunning model: {}".format(this_model))
    # Get TFA and kinetic model indices
    tfa_ix = steady_states_sample

    # Load tfa sample and kinetic parameters into kinetic model
    tfa_sample = pd.read_csv(path_to_tfa_samples, header=0, index_col=0).iloc[tfa_ix]
    parameter_set = parameter_population[this_model]
    kmodel.parameters = parameter_set

    # Load steady-state fluxes and concentrations into the scaffold kmodel
    flux_series = load_fluxes(tfa_sample, tmodel, kmodel,
                         density=DENSITY,
                         ratio_gdw_gww=GDW_GWW_RATIO,
                         concentration_scaling=CONCENTRATION_SCALING,
                         time_scaling=TIME_SCALING)
    
    fluxes = pd.Series([flux_series[reaction.name] for reaction in kmodel.reactions.values()])

    concentrations = load_concentrations(tfa_sample, tmodel, kmodel,
                                         concentration_scaling=CONCENTRATION_SCALING)


    # Fetch equilibrium constants
    load_equilibrium_constants(tfa_sample, tmodel, kmodel,
                               concentration_scaling=CONCENTRATION_SCALING,
                               in_place=True)
    def reset_reactor():
        """
        Function to reset the reactor and load the concentrations and parameters before each simulation
        """
        # Parameterize the rector and initialize with incolum and medium data
        reactor.parametrize(parameter_set, 'strain_1')
        reactor.initialize(concentrations, 'strain_1')
        reactor.initial_conditions['biomass_strain_1'] = 0.037 * 0.05 / 0.28e-12

        for met_ in reactor.medium.keys():
            LC_id = 'LC_' + met_
            LC_id = LC_id.replace('_L', '-L')
            LC_id = LC_id.replace('_D', '-D')
            reactor.initial_conditions[met_] = np.exp(tfa_sample.loc[LC_id]) * 1e9

        # Volume parameters for the reactor
        reactor.models.strain_1.parameters.strain_1_volume_e.value = reactor_volume
        reactor.models.strain_1.parameters.strain_1_cell_volume_e.value = 1.0  # 1.0 #(mum**3)
        reactor.models.strain_1.parameters.strain_1_cell_volume_c.value = 1.0  # 1.0 #(mum**3)
        reactor.models.strain_1.parameters.strain_1_cell_volume_p.value = 1.0  # 1.0 #(mum**3)
        reactor.models.strain_1.parameters.strain_1_volume_c.value = 0.9 * 1.0  # (mum**3)
        reactor.models.strain_1.parameters.strain_1_volume_p.value = 0.1 * 1.0  # (mum**3)
    
    ''' 1. Simulation of trpD9923'''
    reset_reactor()

    start = T.time()
    print("Starting Simulation")
    if hasattr(reactor, 'solver'):
        delattr(reactor, 'solver')

    # Function to stop integration
    def rootfn(t, y, g, user_data):
        t_0 = user_data['time_0']
        t_max = user_data['max_time']
#         print(t)
        curr_t = time.time()
        if (curr_t - t_0) >= t_max:
            g[0] = 0
            print('Did not converge in time')
        else:
            g[0] = 1


    t0 = time.time()
    user_data = {'time_0': t0,
                 'max_time': 60} #2 minutes in seconds

    # Solve the ODEs
    sol_ode_wildtype = reactor.solve_ode(np.linspace(0, TOTAL_TIME, N_STEPS),
        solver_type='cvode',
        rtol=1e-9,
        atol=1e-9,
        max_steps=1e9,
        rootfn=rootfn,
        nr_rootfns=1,
        user_data=user_data)
    end = T.time()
    print("Compelted in {} seconds\n-----------------".format(end - start))

    final_biomass = sol_ode_wildtype.concentrations.iloc[-1]['biomass_strain_1'] * 0.28e-12 / 0.05
    final_anthranilate = sol_ode_wildtype.concentrations.iloc[-1]['anth_e'] * 1e-6 * 136.13
    final_glucose = sol_ode_wildtype.concentrations.iloc[-1]['glc_D_e'] * 1e-6 * 180.156
    print("Final biomass is : {}, final anthranilate is : {}, final glucose is : {}".format(final_biomass,
                                                                                                final_anthranilate,
                                                                                                final_glucose))
    sols_wt.append(sol_ode_wildtype)



# Store all ode solutions
solpop = ODESolutionPopulation(sols_wt, kinetic_models)
solpop.data.to_csv(f'{output_folder}/ode_sols_wt_{repeat}_{generation}.csv')



Running model: 0
Starting Simulation
Did not converge in time
Compelted in 60.01946806907654 seconds
-----------------
Final biomass is : 1.1217708680097498, final anthranilate is : 14.849202119220099, final glucose is : 7.633845706451154e-05

Running model: 1
Starting Simulation
Did not converge in time
Compelted in 60.037012577056885 seconds
-----------------
Final biomass is : 1.17990893926472, final anthranilate is : 13.791897808962373, final glucose is : 2856.55115037884

Running model: 2
Starting Simulation
Compelted in 46.919793128967285 seconds
-----------------
Final biomass is : 1.301868262975232, final anthranilate is : 14.813917469395788, final glucose is : 2608.525963647406

Running model: 3
Starting Simulation
Compelted in 12.616178274154663 seconds
-----------------
Final biomass is : 0.23386761505456993, final anthranilate is : 7.493597440899382, final glucose is : 5665.621361231679

Running model: 4
Starting Simulation
Did not converge in time
Compelted in 60.03796029